# Cargas de abrigados por template - V2

O template loader é uma carga realizada com base no template `import_abrigados_template_v2.googlesheets.md`.
Esse notebook processa dois fluxos, sendo:
- Tratamento, transformação e evolução de dados de abrigados até a persiência no Firestore.
- Criação de novos objetos para enriquecimento em processos como evolução das aplicações, analytcs entre outros.

> **Template file**
>
> template_loader\templates\import_abrigados_template_v2.googlesheets.md


#### Configuração do notebook
Faz import de módulos e configura linter.

In [ ]:
%load_ext blackcellmagic

##### Instalação de dependências


In [ ]:
import re
import os
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import uuid
import sys
from datetime import datetime
from enum import Enum
from selector import environment_selector
from firestore_model_abrigado import AbrigadoEntity, MembroFamiliarEntity, DocumentosEntity, InformacaoAdicionalEntity
from firestore_upload_data import upload_to_firestore, upload_to_firestore_checking_duplicated, update_firestore_single_document
from firestore_query_batch import query_batch
from nome import normalize_nome
from datetime_xlsx import format_xlsx_datetime
from documento import format_documento, is_cpf
from additional_info import format_additional_info, sanitize_additional_info
from guard import guard_pd_notnull
from abrigo_dict import get_abrigo_info

# Legacy entities
from firestore_model_abrigado_legacy import AbrigadoLegacyEntity, MembroFamiliarLegacyEntity, DocumentoLegacyEntity, ResponsavelLegacyEntity

#### Environment Selector
Seleciona manualmente o ambiente de trabalho. O objetivo é evitar erros durante as cargas manuais. Para mais detalhes do ambiente acesse o README.md.

In [ ]:
environment_selector()

### Bootstrap - Configura e carrega ambiente de trabalho


#### Configura chaves e parâmetros
Carrega as variáveis de ambiente e seta o nome da collection de trabalho.

In [ ]:
# Firebase Firestore Config
base_path = os.getenv("SOSRS_BASEPATH")
sosrs_environment = os.getenv("SOSRS_ENVIRONMENT")
sosrs_firestore_keyfile = os.getenv("SOSRS_FIRESTORE_KEYFILE")
sosrs_firestore_path = os.path.join(base_path, sosrs_firestore_keyfile)

# Firestore Collection
collection_name = "abrigado_master"

# Firestore Collection Legacy
collection_name_legacy = "abrigado"

#### Autenticação do notebook
Cria um objeto de conexão com o Firestore do Firebase.

In [ ]:
# Inicialização do Firebase
cred = credentials.Certificate(sosrs_firestore_path)
firebase_admin.initialize_app(cred)
clientFirestore = firestore.client()

## Datasource - Configura fontes externas e coleções estáticas
Configuração de sources e dicionários utilizados no pipeline.

### Configuração de datasources
Define o caminho dos arquivos que serão manupados, nome de abas, dicionários entre outros.

In [ ]:
# Configuração do datasource
datasource_file = "./pipeline/.working/igreja_batista_nacional_de_porto_alegre_nexus_1005_gustavogr_20240510_2032_template.xlsx"
missing_abrigo = "./pipeline/.working/missing_abrigo.json"
master_data_sheet = "abrigados"

## Template - Parametrização do template e estruturas de dados utilizadas
Define parametros e configurações relativos ao template, como nome de colunas, quantidade, entre outros.

#### Nomes de colunas
Mapeia a quantidade de colunas do template e define o nome das colunas utilizadas pelo pandas na importação.

In [ ]:
# Total de colunas
number_of_columns = 79

# Define o número de familiares recebidos por abrigado
numero_de_familiares = 10

# Define o número de propriedades por familiar
numero_de_propriedades_por_familiar = 5

# Define o número de informações adicionais recebidos
numero_de_informacoes_adicionais = 5

# Define o nome das colunas do arquivo de entrada
column_names = [
    "abrigo_nome",
    "abrigo_data_entrada",
    "abrigo_data_saida",
    "abrigado_nome",
    "abrigado_nome_mae",
    "abrigado_data_nascimento",
    "abrigado_idade",
    "abrigado_rg",
    "abrigado_cpf",
    "abrigado_nis",
    "abrigado_titulo_eleitor",
    "abrigado_outros_documentos",
    "abrigado_telefone",
    "abrigado_endereco",
    "abrigado_bairro",
    "abrigado_cidade",
    "abrigado_beneficio_possui",
    "abrigado_beneficio_nome",
    "abrigado_beneficio_valor",
    "abrigado_saude_condicao",
    "abrigado_saude_medicamentos",
    "familiar1_nome",
    "familiar1_data_nascimento",
    "familiar1_idade",
    "familiar1_parentesco",
    "familiar1_documentos",
    "familiar2_nome",
    "familiar2_data_nascimento",
    "familiar2_idade",
    "familiar2_parentesco",
    "familiar2_documentos",
    "familiar3_nome",
    "familiar3_data_nascimento",
    "familiar3_idade",
    "familiar3_parentesco",
    "familiar3_documentos",
    "familiar4_nome",
    "familiar4_data_nascimento",
    "familiar4_idade",
    "familiar4_parentesco",
    "familiar4_documentos",
    "familiar5_nome",
    "familiar5_data_nascimento",
    "familiar5_idade",
    "familiar5_parentesco",
    "familiar5_documentos",
    "familiar6_nome",
    "familiar6_data_nascimento",
    "familiar6_idade",
    "familiar6_parentesco",
    "familiar6_documentos",
    "familiar7_nome",
    "familiar7_data_nascimento",
    "familiar7_idade",
    "familiar7_parentesco",
    "familiar7_documentos",
    "familiar8_nome",
    "familiar8_data_nascimento",
    "familiar8_idade",
    "familiar8_parentesco",
    "familiar8_documentos",
    "familiar9_nome",
    "familiar9_data_nascimento",
    "familiar9_idade",
    "familiar9_parentesco",
    "familiar9_documentos",
    "familiar10_nome",
    "familiar10_data_nascimento",
    "familiar10_idade",
    "familiar10_parentesco",
    "familiar10_documentos",
    "adicional_redes_de_apoio_informacoes",
    "adicional_pet_quantidade",
    "adicional_pet_descricao",
    "adicional_informacao1",
    "adicional_informacao2",
    "adicional_informacao3",
    "adicional_informacao4",
    "adicional_informacao5",
]

#### Enumerador de colunas utilizadas na carga
O objetivo é ter o nome da coluna mapeado em um enum ao invés de declarar uma string. Isso é util para evitar erros no processo, principalmente nos cenários onde é necessario fazer referência a uma coluna.

In [ ]:
class ColumnsNameEnum(Enum):
    ABRIGO_NOME = "abrigo_nome"
    ABRIGO_DATA_ENTRADA = "abrigo_data_entrada"
    ABRIGO_DATA_SAIDA = "abrigo_data_saida"
    ABRIGADO_NOME = "abrigado_nome"
    ABRIGADO_NOME_MAE = "abrigado_nome_mae"
    ABRIGADO_DATA_NASCIMENTO = "abrigado_data_nascimento"
    ABRIGADO_IDADE = "abrigado_idade"
    ABRIGADO_RG = "abrigado_rg"
    ABRIGADO_CPF = "abrigado_cpf"
    ABRIGADO_NIS = "abrigado_nis"
    ABRIGADO_TITULO_ELEITOR = "abrigado_titulo_eleitor"
    ABRIGADO_OUTROS_DOCUMENTOS = "abrigado_outros_documentos"
    ABRIGADO_TELEFONE = "abrigado_telefone"
    ABRIGADO_ENDERECO = "abrigado_endereco"
    ABRIGADO_BAIRRO = "abrigado_bairro"
    ABRIGADO_CIDADE = "abrigado_cidade"
    ABRIGADO_BENEFICIO_POSSUI = "abrigado_beneficio_possui"
    ABRIGADO_BENEFICIO_NOME = "abrigado_beneficio_nome"
    ABRIGADO_BENEFICIO_VALOR = "abrigado_beneficio_valor"
    ABRIGADO_SAUDE_CONDICAO = "abrigado_saude_condicao"
    ABRIGADO_SAUDE_MEDICAMENTOS = "abrigado_saude_medicamentos"
    FAMILIAR1_NOME = "familiar1_nome"
    FAMILIAR1_DATA_NASCIMENTO = "familiar1_data_nascimento"
    FAMILIAR1_IDADE = "familiar1_idade"
    FAMILIAR1_PARENTESCO = "familiar1_parentesco"
    FAMILIAR1_DOCUMENTOS = "familiar1_documentos"
    FAMILIAR2_NOME = "familiar2_nome"
    FAMILIAR2_DATA_NASCIMENTO = "familiar2_data_nascimento"
    FAMILIAR2_IDADE = "familiar2_idade"
    FAMILIAR2_PARENTESCO = "familiar2_parentesco"
    FAMILIAR2_DOCUMENTOS = "familiar2_documentos"
    FAMILIAR3_NOME = "familiar3_nome"
    FAMILIAR3_DATA_NASCIMENTO = "familiar3_data_nascimento"
    FAMILIAR3_IDADE = "familiar3_idade"
    FAMILIAR3_PARENTESCO = "familiar3_parentesco"
    FAMILIAR3_DOCUMENTOS = "familiar3_documentos"
    FAMILIAR4_NOME = "familiar4_nome"
    FAMILIAR4_DATA_NASCIMENTO = "familiar4_data_nascimento"
    FAMILIAR4_IDADE = "familiar4_idade"
    FAMILIAR4_PARENTESCO = "familiar4_parentesco"
    FAMILIAR4_DOCUMENTOS = "familiar4_documentos"
    FAMILIAR5_NOME = "familiar5_nome"
    FAMILIAR5_DATA_NASCIMENTO = "familiar5_data_nascimento"
    FAMILIAR5_IDADE = "familiar5_idade"
    FAMILIAR5_PARENTESCO = "familiar5_parentesco"
    FAMILIAR5_DOCUMENTOS = "familiar5_documentos"
    FAMILIAR6_NOME = "familiar6_nome"
    FAMILIAR6_DATA_NASCIMENTO = "familiar6_data_nascimento"
    FAMILIAR6_IDADE = "familiar6_idade"
    FAMILIAR6_PARENTESCO = "familiar6_parentesco"
    FAMILIAR6_DOCUMENTOS = "familiar6_documentos"
    FAMILIAR7_NOME = "familiar7_nome"
    FAMILIAR7_DATA_NASCIMENTO = "familiar7_data_nascimento"
    FAMILIAR7_IDADE = "familiar7_idade"
    FAMILIAR7_PARENTESCO = "familiar7_parentesco"
    FAMILIAR7_DOCUMENTOS = "familiar7_documentos"
    FAMILIAR8_NOME = "familiar8_nome"
    FAMILIAR8_DATA_NASCIMENTO = "familiar8_data_nascimento"
    FAMILIAR8_IDADE = "familiar8_idade"
    FAMILIAR8_PARENTESCO = "familiar8_parentesco"
    FAMILIAR8_DOCUMENTOS = "familiar8_documentos"
    FAMILIAR9_NOME = "familiar9_nome"
    FAMILIAR9_DATA_NASCIMENTO = "familiar9_data_nascimento"
    FAMILIAR9_IDADE = "familiar9_idade"
    FAMILIAR9_PARENTESCO = "familiar9_parentesco"
    FAMILIAR9_DOCUMENTOS = "familiar9_documentos"
    FAMILIAR10_NOME = "familiar10_nome"
    FAMILIAR10_DATA_NASCIMENTO = "familiar10_data_nascimento"
    FAMILIAR10_IDADE = "familiar10_idade"
    FAMILIAR10_PARENTESCO = "familiar10_parentesco"
    FAMILIAR10_DOCUMENTOS = "familiar10_documentos"
    ADICIONAL_REDES_DE_APOIO_INFORMACOES = "adicional_redes_de_apoio_informacoes"
    ADICIONAL_PET_QUANTIDADE = "adicional_pet_quantidade"
    ADICIONAL_PET_DESCRICAO = "adicional_pet_descricao"
    ADICIONAL_INFORMACAO1 = "adicional_informacao1"
    ADICIONAL_INFORMACAO2 = "adicional_informacao2"
    ADICIONAL_INFORMACAO3 = "adicional_informacao3"
    ADICIONAL_INFORMACAO4 = "adicional_informacao4"
    ADICIONAL_INFORMACAO5 = "adicional_informacao5"
    SEARCH_FIELD_NAME = "search_field_name"

#### Enumeradores e coleçãos para dados Familiares
Define coleções e enumeradores com o nome de colunas de dados familiares para processos de manupulação de dados de familiares.

##### Coleção de colunas de Familiares
Define coleção das colunas utilizadas para coletar informações de familiares.

In [ ]:
FamiliaresColumns = [
    ColumnsNameEnum.FAMILIAR1_NOME.value,
    ColumnsNameEnum.FAMILIAR1_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR1_IDADE.value,
    ColumnsNameEnum.FAMILIAR1_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR1_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR2_NOME.value,
    ColumnsNameEnum.FAMILIAR2_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR2_IDADE.value,
    ColumnsNameEnum.FAMILIAR2_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR2_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR3_NOME.value,
    ColumnsNameEnum.FAMILIAR3_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR3_IDADE.value,
    ColumnsNameEnum.FAMILIAR3_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR3_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR4_NOME.value,
    ColumnsNameEnum.FAMILIAR4_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR4_IDADE.value,
    ColumnsNameEnum.FAMILIAR4_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR4_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR5_NOME.value,
    ColumnsNameEnum.FAMILIAR5_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR5_IDADE.value,
    ColumnsNameEnum.FAMILIAR5_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR5_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR6_NOME.value,
    ColumnsNameEnum.FAMILIAR6_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR6_IDADE.value,
    ColumnsNameEnum.FAMILIAR6_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR6_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR7_NOME.value,
    ColumnsNameEnum.FAMILIAR7_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR7_IDADE.value,
    ColumnsNameEnum.FAMILIAR7_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR7_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR8_NOME.value,
    ColumnsNameEnum.FAMILIAR8_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR8_IDADE.value,
    ColumnsNameEnum.FAMILIAR8_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR8_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR9_NOME.value,
    ColumnsNameEnum.FAMILIAR9_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR9_IDADE.value,
    ColumnsNameEnum.FAMILIAR9_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR9_DOCUMENTOS.value,
    ColumnsNameEnum.FAMILIAR10_NOME.value,
    ColumnsNameEnum.FAMILIAR10_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR10_IDADE.value,
    ColumnsNameEnum.FAMILIAR10_PARENTESCO.value,
    ColumnsNameEnum.FAMILIAR10_DOCUMENTOS.value,
]

##### Enumeradores de constantes de colunas de Familiares
Enumera as constantes utilizadas para laços nas colunas de informações familiares.

In [ ]:
class FamiliaresConstansts(Enum):
    FAMILIAR = "familiar"
    NOME = "nome"
    DATA_NASCIMENTO = "data_nascimento"
    IDADE = "idade"
    PARENTESCO = "parentesco"
    DOCUMENTOS = "documentos"

#### Enumerador de colunas de informações adicionais
Cria um enumerador com as propriedades e informações adicionais que fazem parte do template.

In [ ]:
class InformacaoAdicionalColumns(Enum):
    ADICIONAL_INFORMACAO1 = (ColumnsNameEnum.ADICIONAL_INFORMACAO1.value,)
    ADICIONAL_INFORMACAO2 = (ColumnsNameEnum.ADICIONAL_INFORMACAO2.value,)
    ADICIONAL_INFORMACAO3 = (ColumnsNameEnum.ADICIONAL_INFORMACAO3.value,)
    ADICIONAL_INFORMACAO4 = (ColumnsNameEnum.ADICIONAL_INFORMACAO4.value,)
    ADICIONAL_INFORMACAO5 = (ColumnsNameEnum.ADICIONAL_INFORMACAO5.value,)

#### Enumerador de constantes
Define constantes utilizadas no processo.

In [ ]:
class Constants(Enum):
    ABRIGO = "abrigo"
    '''Usa chave definida no dicionário de abrigos para identificar pessoas sem abrigo'''
    SEM_ABRIGO = "semabrigo"
    """Usa chave definada no dicionário de abrigos para identificar abrigo não informado"""
    ABRIGO_NAO_INFORMADO = "abrigononaoinformadonaimportacao"

#### Colunas para filtro de duplicados
Cria lista com filtros utilizados para verificação de registros duplicados na planilha carregada.

In [ ]:
# Define as colunas para verificar duplicatas
duplicate_columns_filter = [
    ColumnsNameEnum.ABRIGADO_NOME.value,
    ColumnsNameEnum.ABRIGADO_NOME_MAE.value,
    ColumnsNameEnum.ABRIGADO_RG.value,
    ColumnsNameEnum.ABRIGADO_CPF.value,
    ColumnsNameEnum.ABRIGADO_NIS.value,
    ColumnsNameEnum.ABRIGADO_TITULO_ELEITOR.value,
    ColumnsNameEnum.ABRIGADO_OUTROS_DOCUMENTOS.value,
]

#### Coluna de datas para comparações
A forma como excel trata data pode gerar problemas durante o processo de comparação de registros. Para evitar problemas, todas as colunas do tipo data são removidas durante a comparação e posteriormente são adicionadas novamente.

In [ ]:
# Define os campos do tipo data que devem ser tratados
datetime_fields = [
    ColumnsNameEnum.ABRIGO_DATA_ENTRADA.value,
    ColumnsNameEnum.ABRIGO_DATA_SAIDA.value,
    ColumnsNameEnum.ABRIGADO_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR1_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR2_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR3_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR4_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR5_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR6_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR7_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR8_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR9_DATA_NASCIMENTO.value,
    ColumnsNameEnum.FAMILIAR10_DATA_NASCIMENTO.value,
]

## Funções de transformação
Define as funções que serão utilizadas para transformar e/ou osquestrar as transformações no dados.

### Data Quality
Define as funções e estratégias de data quality para o pipeline.

#### Verificação de registros duplicados e manipução
Verifica de forma otimizada, em lotes, se os registros que estão sendo manipulados já estão na collection `abrigado`.

In [ ]:
def filter_duplicated_names(client, collection_name, df_for_filtering):
    # Consulta nomes
    duplicity_check_result = query_batch(client, collection_name, df_for_filtering, ColumnsNameEnum.SEARCH_FIELD_NAME.value, "in", 10)

    # Filtrar para manter apenas entradas com um ID não None
    registered_names = {name: id for name, id in duplicity_check_result.items() if id is not None}

    # Lista de nomes já registrados
    registered_names_list = list(registered_names.keys())

    # Filtrar o DataFrame para remover nomes já registrados
    df_filtered = df_for_filtering[~df_for_filtering[ColumnsNameEnum.SEARCH_FIELD_NAME.value].isin(registered_names_list)]

    return df_filtered

### Transformação
Define as funções de transformação, que são responsáveis por construir entidade e fazer validações, principalmente para garantia de consistência.

> Importante:
>
> O processo de carrregamento do arquivo com o pandas aplicas validações e transformações iniciais, como deduplicação de dados da planilha e aplicação de valores padrões, como por exemplo, nome de abrigo default qunado a propriedade abrigo_nome é vazia ou nula.

#### Construção da entidade Abrigado
A entidade Abrigado é a entidade que representa o documento Abrigado no Firestore do Firebase. Essa entidade e suas propriedades precism estar alinhadas com os times de BI e de aplicação. Importante destacar que em função da velocidade de construção o sistema apresenta fragilidade no tratamento dos dados, por isso é necessário ter atenção redobrada na alteração nos tipos de campos ou rename. Novas propriedades, como por exemplo propriedades para analytics, podem ser adicionadas com segurança.
As alterações na entidade precisam considerar impactos de performance na aplicação durante os fluxos que fazem manipulação de dados no frontend.

##### Construção do Abrigado
Faz a construção e set das propriedades do abrigado, aplicando validações e transformações.

In [ ]:
# Cria uma entidade AbrigadoEntity a partir de uma linha do CSV
def create_abrigado_entity(csv_row):

    # Obtém informações do abrigo
    abrigo_nome, abrigo_id = get_abrigo_info(csv_row[ColumnsNameEnum.ABRIGO_NOME.value])

    # Cria entidade abrigado
    abrigado_entity = AbrigadoEntity(
        abrigo_id=abrigo_id,
        abrigo_nome=abrigo_nome,
        search_field_name=csv_row[ColumnsNameEnum.SEARCH_FIELD_NAME.value],
        abrigado_nome=csv_row[ColumnsNameEnum.ABRIGADO_NOME.value],
        abrigado_nome_mae=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_NOME_MAE.value]),
        abrigado_data_nascimento=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_DATA_NASCIMENTO.value]),
        abrigado_idade=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_IDADE.value]),
        documentos=create_documentos_entity(csv_row),
        abrigado_telefone=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_TELEFONE.value]),
        abrigado_endereco=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_ENDERECO.value]),
        abrigado_bairro=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_BAIRRO.value]),
        abrigado_cidade=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_CIDADE.value]),
        abrigado_beneficio_possui=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_BENEFICIO_POSSUI.value]),
        abrigado_beneficio_nome=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_BENEFICIO_NOME.value]),
        abrigado_beneficio_valor=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_BENEFICIO_VALOR.value]),
        abrigado_saude_condicao=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_SAUDE_CONDICAO.value]),
        abrigado_saude_medicamentos=guard_pd_notnull(csv_row[ColumnsNameEnum.ABRIGADO_SAUDE_MEDICAMENTOS.value]),
        familiares=create_familiar_entities(csv_row),
        adicional_redes_de_apoio_informacoes=guard_pd_notnull(csv_row[ColumnsNameEnum.ADICIONAL_REDES_DE_APOIO_INFORMACOES.value]),
        adicional_pet_quantidade=guard_pd_notnull(csv_row[ColumnsNameEnum.ADICIONAL_PET_QUANTIDADE.value]),
        adicional_pet_descricao=guard_pd_notnull(csv_row[ColumnsNameEnum.ADICIONAL_PET_DESCRICAO.value]),
        adicional_informacoes=create_adicional_informacao_entity(csv_row),
    )

    return abrigado_entity

##### Construção da entidade FamiliarEntity
Membro familiar é a entidade responsável por agragar os familiares do abrigado responsável. O objetivo é registrar um responsável, como por exemplo o pai, e então agregar os familares que estão com ele. Os dados de agregados possui uma quantidade menor de propriedades.

In [ ]:
def create_familiar_entities(df: pd.DataFrame) -> List[MembroFamiliarEntity]:
    familiares = []

    for i in range(1, numero_de_familiares + 1):
        # Define o prefixo para as colunas do familiar que utilizam o padrão familiar1_nome, familiar1_data_nascimento, etc
        prefix = f"{FamiliaresConstansts.FAMILIAR.value}{i}_"

        # Define a propriedade nome que é obrigatória
        nome = df[f"{prefix}{FamiliaresConstansts.NOME.value}"]

        if guard_pd_notnull(nome) != None:

            # Cria entidade familiar
            familiar = MembroFamiliarEntity(
                nome=nome,
                data_nascimento=guard_pd_notnull(df[f"{prefix}{FamiliaresConstansts.DATA_NASCIMENTO.value}"]),
                idade=guard_pd_notnull(df[f"{prefix}{FamiliaresConstansts.IDADE.value}"]),
                parentesco=guard_pd_notnull(df[f"{prefix}{FamiliaresConstansts.PARENTESCO.value}"]),
                documentos=guard_pd_notnull(df[f"{prefix}{FamiliaresConstansts.DOCUMENTOS.value}"]),
            )

            # Verifica se documento é um cpf
            documento = guard_pd_notnull(df[f"{prefix}{FamiliaresConstansts.DOCUMENTOS.value}"])
            if is_cpf(documento):
                familiar.documentos = DocumentosEntity(cpf=documento)
            else:
                familiar.documentos = DocumentosEntity(outros_documentos=documento) if documento else None

            familiares.append(familiar)

    return familiares if len(familiares) > 0 else None

##### Construção da entidade Documentos e append na entidade Abrigado
A entida documentos é responsável por agregar os documentos do Abrigado. Ela é uma entidade para não somente agregar documentos mas para facilitar a sua refatoração no futuro para aplicar estratégias de pseudo-anonimização em função da LGPD.

In [ ]:
# Cria uma entidade DocumentosEntity a partir de uma linha do CSV
def create_documentos_entity(data: dict) -> DocumentosEntity:
    kargs = {}

    if pd.notna(data.get(ColumnsNameEnum.ABRIGADO_RG.value)):
        kargs["rg"] = data.get(ColumnsNameEnum.ABRIGADO_RG.value)
    if pd.notna(data.get(ColumnsNameEnum.ABRIGADO_CPF.value)):
        kargs["cpf"] = data.get(ColumnsNameEnum.ABRIGADO_CPF.value)
    if pd.notna(data.get(ColumnsNameEnum.ABRIGADO_NIS.value)):
        kargs["nis"] = data.get(ColumnsNameEnum.ABRIGADO_NIS.value)
    if pd.notna(data.get(ColumnsNameEnum.ABRIGADO_TITULO_ELEITOR.value)):
        kargs["titulo_eleitor"] = data.get(ColumnsNameEnum.ABRIGADO_TITULO_ELEITOR.value)
    if pd.notna(data.get(ColumnsNameEnum.ABRIGADO_OUTROS_DOCUMENTOS.value)):
        kargs["outros_documentos"] = data.get(ColumnsNameEnum.ABRIGADO_OUTROS_DOCUMENTOS.value)

    return DocumentosEntity(**kargs)

##### Construção das Informações Adicionais
Gera a coleção com as informações adicionais encontradas.

In [ ]:
def create_adicional_informacao_entity(df: pd.DataFrame) -> list[InformacaoAdicionalEntity]:
    adicional_informacao = []
    for field in InformacaoAdicionalColumns:
        if pd.notna(df[field.value[0]]) and df[field.value[0]] != "":
            adicional_informacao.append(
                InformacaoAdicionalEntity(
                    chave=field.name,
                    valor=df[field.value[0]],
                )
            )

    return adicional_informacao

#### Pipeline de transformação
O pipeline de transformação é responsável por orquestrar a construção das entidades, principalmente apoiando na consistência. 
Qualquer alteração nesse fluxo deve ser avaliada com muito critério para não impactar as importações.

In [ ]:
def transform_csv_data(csv_row):

    # Cria entidade abrigado
    abrigado_entity = create_abrigado_entity(csv_row)

    # Adiciona familiares
    abrigado_entity.familiares = create_familiar_entities(csv_row)

    # Adiciona informações adicionais
    abrigado_entity.additional_info = create_adicional_informacao_entity(csv_row)

    return abrigado_entity

#### Persistência legada
Esse pipeline é utilizado para persistir as entidades legadas na collection principal, `abrigado`. Essa collection é tratada como legada em função da sua modelagem e por não tratar todas as informações enviadas no template `import_abrigados_template_v2.googlesheets.md`. Esse fluxo é responsável apenas pela criação das entidades, uma vez que o dado já foi tratado no Pipeline de transformação.

In [ ]:
search_field_name_legacy = "search_field_name"
abrigo_nome_legacy = None


# Cria uma entidade AbrigadoLegacyEntity a partir de uma linha do CSV
def create_abrigado_entity_legacy(csv_row):

    abrigo_nome_legacy, abrigo_id_legacy = get_abrigo_info(csv_row[ColumnsNameEnum.ABRIGO_NOME.value])

    # Se abrigo for vazio ou NaN, não importaremos
    if abrigo_id_legacy is None:
        if csv_row[ColumnsNameEnum.ABRIGO_NOME.value] == "" or pd.isna(csv_row[ColumnsNameEnum.ABRIGO_NOME.value]):
            sys.exit(f"Erro: Abrigo não encontrado para o nome {csv_row[ColumnsNameEnum.ABRIGO_NOME.value]}")
        else:
            abrigo_id_legacy = None

    abrigado_entity_legacy = AbrigadoLegacyEntity(
        nome=csv_row[ColumnsNameEnum.ABRIGADO_NOME.value],
        search_field_name=normalize_nome(csv_row[ColumnsNameEnum.ABRIGADO_NOME.value]),
        dataNascimento=(
            csv_row[ColumnsNameEnum.ABRIGADO_DATA_NASCIMENTO.value] if pd.notna(csv_row[ColumnsNameEnum.ABRIGADO_DATA_NASCIMENTO.value]) else None
        ),
        endereco=csv_row[ColumnsNameEnum.ABRIGADO_ENDERECO.value] if pd.notna(csv_row[ColumnsNameEnum.ABRIGADO_ENDERECO.value]) else None,
        temDocumento=None,
        acompanhadoMenor=None,
        temRenda=None,
        renda=None,
        temHabitacao=None,
        situacaoMoradia=None,
        necessidadesImediatas=None,
        cadastradoCadUnico=None,
        abrigoId=abrigo_id_legacy,
        abrigoNome=abrigo_nome_legacy,
        documentos=[format_documento(csv_row[ColumnsNameEnum.ABRIGADO_CPF.value])],
        additional_info=[
            format_additional_info(ColumnsNameEnum.ABRIGO_DATA_ENTRADA.value, csv_row[ColumnsNameEnum.ABRIGO_DATA_ENTRADA.value]),
            format_additional_info(ColumnsNameEnum.ABRIGADO_TELEFONE.value, csv_row[ColumnsNameEnum.ABRIGADO_TELEFONE.value]),
            format_additional_info(ColumnsNameEnum.ABRIGADO_BAIRRO.value, csv_row[ColumnsNameEnum.ABRIGADO_BAIRRO.value]),
            format_additional_info(ColumnsNameEnum.ABRIGADO_CIDADE.value, csv_row[ColumnsNameEnum.ABRIGADO_CIDADE.value]),
            format_additional_info(ColumnsNameEnum.ABRIGADO_BENEFICIO_NOME.value, csv_row[ColumnsNameEnum.ABRIGADO_BENEFICIO_NOME.value]),
            format_additional_info(
                ColumnsNameEnum.ADICIONAL_REDES_DE_APOIO_INFORMACOES.value, csv_row[ColumnsNameEnum.ADICIONAL_REDES_DE_APOIO_INFORMACOES.value]
            ),
            format_additional_info(ColumnsNameEnum.ABRIGADO_SAUDE_CONDICAO.value, csv_row[ColumnsNameEnum.ABRIGADO_SAUDE_CONDICAO.value]),
            format_additional_info(ColumnsNameEnum.ABRIGADO_SAUDE_MEDICAMENTOS.value, csv_row[ColumnsNameEnum.ABRIGADO_SAUDE_MEDICAMENTOS.value]),
        ],
    )

    # Remove strings vazias
    abrigado_entity_legacy.additional_info = sanitize_additional_info(abrigado_entity_legacy.additional_info)

    # Retorna a entidade abrigado
    return abrigado_entity_legacy


# Cria a entidade membro familiar e adiciona ao abrigado
def append_membro_familiar_entity_legacy(nome, data_nascimento, grau_parentesco, abrigado_entity_legacy):

    # Criar um dicionário com valores válidos apenas
    kwargs = {}
    if pd.notna(nome) and str(nome).strip() != "":
        kwargs["nome"] = str(nome).strip()
    if pd.notna(data_nascimento) and str(data_nascimento).strip() != "":
        kwargs["data_nascimento"] = str(data_nascimento).strip()
    if pd.notna(grau_parentesco) and str(grau_parentesco).strip() != "":
        kwargs["grau_parentesco"] = str(grau_parentesco).strip()

    # Verifica se há pelo menos um campo válido
    if kwargs:
        membro_familiar = MembroFamiliarLegacyEntity(**kwargs)
        # Adiciona membro familiar à lista de membros familiares se existir a lista
        if hasattr(abrigado_entity_legacy, "grupoFamiliar"):
            if isinstance(abrigado_entity_legacy.grupoFamiliar, list):
                abrigado_entity_legacy.grupoFamiliar.append(membro_familiar)
            else:
                abrigado_entity_legacy.grupoFamiliar = [membro_familiar]
        else:
            abrigado_entity_legacy.grupoFamiliar = [membro_familiar]

    return abrigado_entity_legacy


# Cria a entidade responsável e adiciona ao abrigado
def append_responsavel_entity_legacy(nome, data_nascimento, abrigado_entity_legacy):
    responsavel = ResponsavelLegacyEntity(
        nome=nome if pd.notna(nome) and nome != "" else None,
        data_nascimento=data_nascimento if pd.notna(data_nascimento) and data_nascimento != "" else None,
        additional_info=None,
    )
    abrigado_entity_legacy.responsavel = responsavel
    return abrigado_entity_legacy


# Cria a entidade documento e adiciona ao abrigado
def append_documento_entity_legacy(cpf, rg, nis, titulo_eleitor, outros_documentos, abrigado_entity_legacy):
    documento_entity = DocumentoLegacyEntity(
        cpf=cpf if pd.notna(cpf) and cpf != "" else None,
        rg=rg if pd.notna(rg) and rg != "" else None,
        nis=nis if pd.notna(nis) and nis != "" else None,
        titulo_eleitor=titulo_eleitor if pd.notna(titulo_eleitor) and titulo_eleitor != "" else None,
        outros_documentos=outros_documentos if pd.notna(outros_documentos) else None,
    )
    abrigado_entity_legacy.responsavel.documentos = documento_entity
    return abrigado_entity_legacy


# Cria a entidade abrigado legada e adiciona membros familiares, responsável e documentos
def build_abrigado_legacy_entity(csv_row):
    # Cria entidade abrigado
    abrigado_entity_legacy = create_abrigado_entity_legacy(csv_row)

    for i in range(1, 11):
        # Adiciona membros familiares se existirem
        prefix = f"{FamiliaresConstansts.FAMILIAR.value}{i}_"

        # Adiciona membro familiar
        append_membro_familiar_entity_legacy(
            csv_row.get(f"{prefix}{FamiliaresConstansts.NOME.value}"),
            csv_row.get(f"{prefix}{FamiliaresConstansts.DATA_NASCIMENTO.value}"),
            csv_row.get(f"{prefix}{FamiliaresConstansts.PARENTESCO.value}"),
            abrigado_entity_legacy,
        )

    # Adiciona responsável
    append_responsavel_entity_legacy(
        csv_row[ColumnsNameEnum.ABRIGADO_NOME.value], csv_row[ColumnsNameEnum.ABRIGADO_DATA_NASCIMENTO.value], abrigado_entity_legacy
    )

    # Adiciona documentos
    append_documento_entity_legacy(
        csv_row[ColumnsNameEnum.ABRIGADO_CPF.value],
        csv_row[ColumnsNameEnum.ABRIGADO_RG.value],
        csv_row[ColumnsNameEnum.ABRIGADO_NIS.value],
        csv_row[ColumnsNameEnum.ABRIGADO_TITULO_ELEITOR.value],
        csv_row[ColumnsNameEnum.ABRIGADO_OUTROS_DOCUMENTOS.value],
        abrigado_entity_legacy,
    )

    return abrigado_entity_legacy

## Execução

#### Abrigos não mapeados
Para facilitar o processo de integração e não ficar impacto pela falta de normalização do nome dos abrigos, existe um mapeamento de abrigos entre o que estão nas planilhas e o que está cadastrado no sistema.
Durante a importação, quando um abrigo não é encontrado no mapeamento, então ele é excluído do dataframe de processamento e é persistido em `missing_abrigo.json`, setado na seção `Configuração de Datasorces`.


#### Carrega planilha e executa o fluxo do pipeline.
Os dados da planilha recebida são tratados nesse fluxo inicial, onde são removidoas linhas duplicadas, formatação de data, validações básicas, como por exemplo a existência do nome do abrigado, entre outras funções.
Uma vez aplicado essas transformações iniciais, os dados são evoluidos para os processos de data quality, transformação e persistência.

In [ ]:
# Carrega o arquivo xlsx agregador
if os.path.exists(datasource_file):
    xlsx = pd.ExcelFile(datasource_file)
else:
    print(f"Datasource file not found. {datasource_file}")
    sys.exit(1)

# Define memória para abrigos não encon trados
missing_abrigos = pd.DataFrame()

try:
    # Faz a leitura da aba master_data_sheet
    data_source = pd.read_excel(xlsx, sheet_name=master_data_sheet, header=2, names=column_names, usecols=range(number_of_columns))

    # Remove linhas onde a coluna 'nome' é NaN ou vazia
    data_source = data_source[data_source[ColumnsNameEnum.ABRIGADO_NOME.value].notna() & (data_source[ColumnsNameEnum.ABRIGADO_NOME.value] != "")]

    if data_source.empty:
        print("No valid data found in the master sheet. The field Nome is mandatory. Exiting...")
        sys.exit(1)

    # Formata data para compatbilidade com excel, principalmente para datas com ano menor que 1900 e serialização de datas
    format_xlsx_datetime(data_source, datetime_fields)

    # Identifica duplicatas, mantendo a primeira ocorrência
    duplicates = data_source[data_source.duplicated(subset=duplicate_columns_filter, keep=False)]

    # Filtra para manter apenas as duplicatas excluídas (ignora a primeira ocorrência)
    duplicates_excluded = duplicates.drop_duplicates(subset=duplicate_columns_filter, keep="first")

    # Remove linhas duplicadas baseado nas colunas especificadas no DataFrame original
    data_source = data_source.drop_duplicates(subset=duplicate_columns_filter, keep="first")

    if not duplicates_excluded.empty:
        print("Registros duplicados que foram excluídos:")
        print(duplicates_excluded)

    # Verifica se 'abrigo' é vazia ou NaN e 'responsavel_nome' é válida; se sim, define 'abrigo' como "Abrigo não informado na importação"
    data_source.loc[
        (data_source[ColumnsNameEnum.ABRIGO_NOME.value].isna() | (data_source[ColumnsNameEnum.ABRIGO_NOME.value] == ""))
        & data_source[ColumnsNameEnum.ABRIGADO_NOME.value].notna()
        & (data_source[ColumnsNameEnum.ABRIGADO_NOME.value] != ""),
        ColumnsNameEnum.ABRIGO_NOME.value,
    ] = Constants.ABRIGO_NAO_INFORMADO.value

    # Adiciona uma coluna para verificar a existência do abrigo usando get_abrigo_info
    data_source["abrigo_exists"] = data_source[ColumnsNameEnum.ABRIGO_NOME.value].apply(lambda x: get_abrigo_info(x)[1] is not None)

    # Define colunas para comparação de duplicatas, mantendo todas as colunas originais, exceto a coluna de busca específica
    comparison_columns = [col for col in data_source.columns if col not in datetime_fields and col != ColumnsNameEnum.SEARCH_FIELD_NAME.value]

    # Identifica registros duplicados usando as colunas definidas para comparação
    data_source["duplicate"] = data_source.duplicated(subset=comparison_columns, keep=False)

    # Filtra para encontrar registros válidos: não duplicados e com abrigo existente
    valid_dataframe = data_source[(data_source["abrigo_exists"]) & (~data_source["duplicate"])]

    # Identifica registros com abrigos ausentes
    missing_abrigos = data_source[~data_source["abrigo_exists"]]

    if not missing_abrigos.empty:
        print(f"Missing abrigos found: {missing_abrigos[ColumnsNameEnum.ABRIGO_NOME.value].unique()}")
        print("Removing missing abrigos from new records.")
        # Salvar os abrigos ausentes para arquivo JSON (substitua 'missing_abrigo' pela variável correta com o caminho do arquivo)
        missing_abrigos.to_json(missing_abrigo, orient="records", lines=True)

        if valid_dataframe.empty:
            print("All records have missing abrigos. Exiting...")
            sys.exit(1)

    # Atualiza data_source para conter apenas registros válidos
    data_source = valid_dataframe.copy()

    # Opcionalmente, pode-se remover as colunas 'abrigo_exists' e 'duplicate' se não forem mais necessárias
    data_source.drop(columns=["abrigo_exists", "duplicate"], inplace=True)

    # Cria coluna search_field_nome
    data_source[ColumnsNameEnum.SEARCH_FIELD_NAME.value] = data_source.apply(
        lambda row: normalize_nome(row[ColumnsNameEnum.ABRIGADO_NOME.value]), axis=1
    )

except ValueError:
    print(f"Error reading sheets. Check the aggreate spreadsheet in the documentation. {master_data_sheet}")
    sys.exit(1)


# Transforma os dados, valida e faz o upload
try:
    # Persiste as entidades ricas no Firestore
    upload_to_firestore_checking_duplicated(clientFirestore, collection_name, data_source, transform_csv_data, filter_duplicated_names)

    # Persiste as entidades legadas no Firestore
    upload_to_firestore_checking_duplicated(
        clientFirestore, collection_name_legacy, data_source, build_abrigado_legacy_entity, filter_duplicated_names
    )

    print("Upload finished.")
except Exception as e:
    print(f"Error uploading data to Firestore: {e}")
    sys.exit(1)

print("Files processing finished.")